In [1]:
import torch
import matplotlib

import torch.nn as nn
import torch.optim as optim

The architecture of the MLP used consist of 1 input layer which takes two numbers as input , then 4 hidden layers amd lastly an output layer which delivers the output. The sizes of the layers can be seen as first being increased to try to form a more abstract representation at the beginning layers, after that the size of layers can be seen as decreasing to represent more the essential features that have an influence on our output. ReLU is used after every layer except the last one to introduce non-linearity into the model because it is very needed as multiplication is non linear.

The model is intialized at the end.

In [7]:
class MLP(nn.Module):

  def __init__(self):

    super(MLP,self).__init__()

    self.linear1 = nn.Linear(2,32)

    self.relu = nn.ReLU()

    self.linear2 = nn.Linear(32,64)

    self.linear3 = nn.Linear(64,128)

    self.linear4 = nn.Linear(128,64)

    self.linear5 = nn.Linear(64,32)

    self.linear6 = nn.Linear(32,1)


  def forward(self,x):

    x = self.linear1(x)

    x = self.relu(x)

    x = self.linear2(x)

    x = self.relu(x)

    x = self.linear3(x)

    x = self.relu(x)

    x = self.linear4(x)

    x = self.relu(x)

    x = self.linear5(x)

    x = self.relu(x)

    x = self.linear6(x)


    return x

model = MLP()

first a random seed is set that make sure that operation of generating random is determenistic, after that we generate the first half of our training data by using torch.rand to create a tensor of shape (1000,2) that contain random values from the range 0 to 1

then we construct the y data for this tensor by calculating the products of the two colummns for every row, unnsqueeze method is used to reshape the result into the format that pytorch expects.

the second part of the training data is constructed the same way as the first part, but the difference is values inside it lies in the range from 4 to 6.

torch.cat is used to concatenate the two tensors into one tensor which will be the training data for our model, this is done for both X data and y data.

finally the data is shuffled to make sure that the data is mixed well preventing any bias during training.

so our training data will have two number from 0 to 1 and numbers from 4 to 6 only.

In [25]:
torch.manual_seed(42)
X_train1= torch.rand(1000, 2) * (1 - 0) + 0
y_train1 = (X_train1[:, 0] * X_train1[:, 1]).unsqueeze(1)
print(X_train1.min(),X_train1.max())


X_train2= torch.rand(1000, 2) * (6 - 4) + 4
y_train2 = (X_train2[:, 0] * X_train2[:, 1]).unsqueeze(1)
print(X_train2.min(),X_train2.max())


X_train = torch.cat((X_train1, X_train2), dim=0)
y_train = torch.cat((y_train1, y_train2), dim=0)


shuffle_idx = torch.randperm(X_train.shape[0])
X_train = X_train[shuffle_idx]
y_train = y_train[shuffle_idx]


tensor(0.0023) tensor(0.9998)
tensor(4.0035) tensor(5.9978)


the loss function that is used is Mean Squared Error, it is a very common loss function for regression, and its main benefit that it penalizez more larger errors making the model more robust. Adam was used as an optimiser because it is an adaptive learning rate optimization algorithm which is based on the methodologies of momentum and root mean squared propagation, 0.001 is the common choice of learning rate for Adam.

for every epoch, the model generate predictions for the training dataset, then the loss is predicted between the predictions and the actual values, gradients are cleared so they dont accumulate, then the derivative of the loss with respect to the gradient is computed. then the optimzer updates the parameters based on the gradient and learning rate

In [21]:
epochs = 10000

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr= 0.001)
loss_history = []
for epoch in range(epochs):

  y_predict = model(X_train)

  loss = criterion(y_predict,y_train)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  loss_history.append(loss.item())
  if not epoch % 200:
      # Print out the loss every 10 iterations
      print('epoch {}, loss {}'.format(epoch, loss.item()))



epoch 0, loss 318.1556701660156
epoch 200, loss 1.1489644050598145
epoch 400, loss 0.01569806970655918
epoch 600, loss 0.011223483830690384
epoch 800, loss 0.0011264366330578923
epoch 1000, loss 0.0013750676298514009
epoch 1200, loss 0.0014645884511992335
epoch 1400, loss 0.0005914957146160305
epoch 1600, loss 0.007005619816482067
epoch 1800, loss 0.005941986106336117
epoch 2000, loss 0.0020460367668420076
epoch 2200, loss 0.028876449912786484
epoch 2400, loss 0.006020625587552786
epoch 2600, loss 0.0046911160461604595
epoch 2800, loss 0.0025117432232946157
epoch 3000, loss 0.005613299552351236
epoch 3200, loss 0.0007305254694074392
epoch 3400, loss 0.0019731612410396338
epoch 3600, loss 0.009693525731563568
epoch 3800, loss 0.037666816264390945
epoch 4000, loss 0.005476364400237799
epoch 4200, loss 0.0071317111141979694
epoch 4400, loss 0.0006999404286034405
epoch 4600, loss 0.0008855409105308354
epoch 4800, loss 0.011164985597133636
epoch 5000, loss 0.004288727883249521
epoch 5200, l

random samples in range of the training data is generated. just like the training dataset, they are done the same way, the first half contains data from range 1 to 0 and their product result. the second half contain data from range from 4 to 6 and their target values, then the two halves are concatenated.

then we switch the model to evaluation mode, then we use torch.no_grad as during evaluation back propagation is not needed, then we predict our evaluation data, and we calculate the mean squared error between the predictions and the actual values.

In [22]:
torch.manual_seed(43)
X_eval1= torch.rand(500, 2) * (1 - 0) + 0
y_eval1 = (X_eval1[:, 0] * X_eval1[:, 1]).unsqueeze(1)

X_eval2= torch.rand(500, 2) * (6 - 4) + 4
y_eval2 = (X_eval2[:, 0] * X_eval2[:, 1]).unsqueeze(1)

X_eval = torch.cat((X_eval1, X_eval2), dim=0)
y_eval = torch.cat((y_eval1, y_eval2), dim=0)

model.eval()
with torch.no_grad():
    eval_predictions = model(X_eval)
    eval_error = torch.mean(torch.abs(eval_predictions - y_eval))


print(f"Mean Absolute Error on evaluation Data: {eval_error.item()}")



Mean Absolute Error on Training Data: 0.01415469590574503


Random samples outside the range of training data are constructed, the torch.rand is used to construct a tensor with shape (500,2) with values in the range of 1 to 3 with their product values. then we predict our test data, and calculate the mean squared error between the predictions and the actual values.



In [23]:
X_test = torch.rand(500, 2) * (3 - 1) + 1
print(X_test.min(),X_test.max())
y_test = (X_test[:, 0] * X_test[:, 1]).unsqueeze(1)
test_predictions = model(X_test)
test_error = torch.mean(torch.abs(test_predictions - y_test))
print(f"Mean Absolute Error on Test Data: {test_error.item()}")

tensor(1.0001) tensor(2.9965)
Mean Absolute Error on Test Data: 0.9088492393493652
